In [3]:
from LsegNet import LitLseg
from Lseg.utils.util import get_dataset, get_labels

In [ ]:
# Define model
labels = get_labels()

config = {
    "max_epochs": 50,
    "num_classes": len(labels),
    "batch_size": 2,
    "base_lr": 0.04,
}

# load model 
load_checkpoint_path = r'checkpoints/checkpoint_epoch=0-val_loss=4.7199.ckpt'
model = LitLseg.load_from_checkpoint(
    load_checkpoint_path,
    max_epochs = config['max_epochs'],
    num_classes = config['num_classes'],
    batch_size = config['batch_size'],
    base_lr = config['base_lr'],
)

model = model.to(device='cuda').float()

/home/rotakagui/miniconda3/envs/CLIP/lib/python3.8/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [ ]:
# Load evaluation ade20k dataset 
from torch.utils.data import DataLoader

val_ade20k_dataset = get_dataset(dataset_name="ade20k", get_train=False)
val_dataloaders = DataLoader(val_ade20k_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=8)

	Mapping ade20k-150 -> universal
	Mapping bdd -> universal
	Mapping cityscapes-19 -> universal
	Mapping coco-panoptic-133 -> universal
	Mapping idd-39 -> universal
	Mapping mapillary-public65 -> universal
	Mapping sunrgbd-37 -> universal
	Mapping ade20k-150-relabeled -> universal
	Mapping bdd-relabeled -> universal
	Mapping cityscapes-19-relabeled -> universal
	Mapping cityscapes-34-relabeled -> universal
	Mapping coco-panoptic-133-relabeled -> universal
	Mapping idd-39-relabeled -> universal
	Mapping mapillary-public65-relabeled -> universal
	Mapping sunrgbd-37-relabeled -> universal

	Creating 1x1 conv for test datasets...
Totally 2000 samples in val set.
Checking image&label pair val list done!
image folder path: data/mseg_dataset/ADE20K/ADEChallengeData2016/
text path: mseg-api/mseg/dataset_lists/ade20k-150-relabeled/list/val.txt


In [9]:
# Define metrics 
from torchmetrics import Accuracy, JaccardIndex

NUM_CLASSES = 195
accuracy_fn = Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(device="cuda")
iou_fn = JaccardIndex(task="multiclass", num_classes=NUM_CLASSES).to(device="cuda")

In [10]:
# Evaluate accuracy IoU
import torch
import tqdm

mean_accuracy = 0.0
mean_iou = 0.0

model.eval()
with torch.no_grad():
    for img, annotation in tqdm.tqdm(val_dataloaders):
        img = img.to(device='cuda').float()
        annotation = annotation.to(device='cuda').float()
        output = model(img)
        prediction = torch.argmax(output,dim=1)
        mean_accuracy += accuracy_fn(prediction, annotation).item()
        mean_iou += iou_fn(prediction,annotation).item()

mean_accuracy = mean_accuracy / len(val_dataloaders)
mean_iou = mean_iou / len(val_dataloaders)
print(f"Number of examples: {len(val_ade20k_dataset)}")
print(f"Accuracy: {mean_accuracy}")
print(f"IoU: {mean_iou}")


100%|██████████| 1000/1000 [03:00<00:00,  5.53it/s]

Number of examples: 2000
Accuracy: 0.6008381535221125
IoU: 0.19140128025584272
